## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-19-17-07-21 +0000,nypost,Chaos at major AI summit as scandal-plagued Bi...,https://nypost.com/2026/02/19/business/bill-ga...
1,2026-02-19-17-07-00 +0000,wsj,Trump Hints at 10-Day Window for Decision on Iran,https://www.wsj.com/world/middle-east/trump-hi...
2,2026-02-19-17-06-20 +0000,nyt,Live Updates: Former Prince Andrew Arrested in...,https://www.nytimes.com/live/2026/02/19/world/...
3,2026-02-19-17-05-47 +0000,nypost,West Virginia accuses Apple of allowing iCloud...,https://nypost.com/2026/02/19/business/west-vi...
4,2026-02-19-17-04-34 +0000,nypost,"Malibu sues LA, California over Palisades fire...",https://nypost.com/2026/02/19/us-news/malibu-s...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2569/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
18,trump,36
31,epstein,18
28,andrew,18
23,iran,18
48,california,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
132,2026-02-19-11-48-00 +0000,wsj,"As Trump’s Board of Peace meets Thursday, can ...",https://www.wsj.com/politics/trump-moves-towar...,119
2,2026-02-19-17-06-20 +0000,nyt,Live Updates: Former Prince Andrew Arrested in...,https://www.nytimes.com/live/2026/02/19/world/...,93
119,2026-02-19-12-11-49 +0000,nypost,UK Prime Minister Keir Starmer warned ‘nobody ...,https://nypost.com/2026/02/19/world-news/uk-pr...,92
43,2026-02-19-15-57-00 +0000,wsj,Former Prince Andrew Arrested as Police Probe ...,https://www.wsj.com/world/uk/former-prince-and...,90
358,2026-02-18-19-06-00 +0000,wsj,NIH Director Dr. Jay Bhattacharya will step in...,https://www.wsj.com/politics/policy/rfk-jr-cdc...,90


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
132,119,2026-02-19-11-48-00 +0000,wsj,"As Trump’s Board of Peace meets Thursday, can ...",https://www.wsj.com/politics/trump-moves-towar...
2,93,2026-02-19-17-06-20 +0000,nyt,Live Updates: Former Prince Andrew Arrested in...,https://www.nytimes.com/live/2026/02/19/world/...
186,66,2026-02-19-09-47-36 +0000,cbc,South Korea's ex-president Yoon receives life ...,https://www.cbc.ca/news/world/yoon-suk-yeol-li...
75,60,2026-02-19-14-39-21 +0000,nypost,White House warns Iran it’d be ‘very wise’ to ...,https://nypost.com/2026/02/19/us-news/white-ho...
183,56,2026-02-19-09-58-53 +0000,nypost,Rep. Lauren Boebert’s son Tyler slapped with a...,https://nypost.com/2026/02/19/us-news/lauren-b...
358,54,2026-02-18-19-06-00 +0000,wsj,NIH Director Dr. Jay Bhattacharya will step in...,https://www.wsj.com/politics/policy/rfk-jr-cdc...
349,53,2026-02-18-19-43-00 +0000,wsj,Fed officials signaled little appetite for red...,https://www.wsj.com/economy/central-banking/fe...
351,52,2026-02-18-19-30-00 +0000,cbc,"8 skiers found dead, 1 still missing in deadli...",https://www.cbc.ca/news/world/california-avala...
221,46,2026-02-19-05-01-09 +0000,nyt,Policy Flip-Flops Hurt the British Leader. The...,https://www.nytimes.com/2026/02/19/world/europ...
185,38,2026-02-19-09-49-00 +0000,wsj,"Tech, Media & Telecom Roundup: Market Talk",https://www.wsj.com/business/tech-media-teleco...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
